In [16]:
# Libraries
import base64
from requests import post, get
import json
import pandas as pd
import numpy as np
import time

In [11]:
# Read credentials from JSON config file
config_file = (open('Spotify_API_Credentials.json'))
config_file_json = json.load(config_file)

client = config_file_json['credentials'][0]['client']
client_sec = config_file_json['credentials'][0]['client_sec']
id_playlist = config_file_json['params'][0]['id_playlist']

config_file.close
del(config_file)

In [12]:
# Get token for the Spotify API
def get_token(client, client_sec):
    auth_string = client + ':' + client_sec
    auth_bytes = auth_string.encode("utf-8")
    auth_base_64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = 'https://accounts.spotify.com/api/token'
    headers =  {
        "Authorization" : "Basic " + auth_base_64,
        "Content-Type" : "application/x-www-form-urlencoded"
    }

    data = {"grant_type" : "client_credentials"}

    result = post(url, headers=headers, data=data)
    token = json.loads(result.content)['access_token']

    return token

def get_auth_headers(token):
    return {"Authorization" : "Bearer " + token}

token = get_token(client, client_sec)

In [13]:
def get_propertiesjson(in_properties=None):
    # Diccionario con elementos del json que me ha salido del nabo identificar XD    
    properties = {
        "nombre" : "cancion['track']['name']",
        "artista" : "cancion['track']['artists'][0]['name']",
        "addedin_date" : "cancion['added_at'][:10]",
        "addedin_hour" : "cancion['added_at'][11:-1]",
        "addedby" : "cancion['added_by']['id']",
        "cover" : "cancion['track']['album']['images'][0]['url']",
        "fec_lanz" : "cancion['track']['album']['release_date']",
        "duracion" : "cancion['track']['duration_ms'] / 10**3",
        "popularity" : "cancion['track']['popularity']",
        "explicit" : "cancion['track']['explicit']"
    }

    # Caso de todas las variables (sin especificar el parámetro)
    if in_properties is None:
        in_properties = properties.keys()

    # Match con los elementos de entrada
    identified = []
    for prop_in in in_properties:
        if prop_in in list(properties.keys()):
            identified.append(prop_in)
        else:
            print("La propiedad '" +  prop_in + "' no ha sido identificada. Se continuará con el proceso ignorándola.")

    # Filtrado para propiedades identificadas
    out_properties = {key: properties[key] for key in identified}

    return out_properties

In [18]:
def canciones_playlist(token, id_playlist, properties=None):

    # Creación de offset por la puta paginación de mierda de la API
    offset = 0
    step = 100
    i = 0

    # Creación de diccionario final con las propiedades especificadas
    result_dict = {}

    # Llamada a diccionario de configuración para cada propiedad del json deseada
    out_properties = get_propertiesjson(properties)

    # Loop para cada request por la puta paginación de mierda de la API
    while True:
        
        # Cronómetro
        start_time = time.time()
    
        # Petición
        url = 'https://api.spotify.com/v1/playlists/' + id_playlist + f'/tracks?offset={offset}&limit={step}'
        headers = get_auth_headers(token)
        result = get(url, headers=headers)
        result_json = json.loads(result.content)

        # Cada cancion del json
        for cancion in result_json["items"]:
            # Cada subpropiedad del json
            for propiedad in out_properties.keys():
                if propiedad not in result_dict:
                    result_dict[propiedad] = []
                try:
                    result_dict[propiedad].append(eval(out_properties[propiedad]))
                except:
                    result_dict[propiedad].append(float("nan"))

        # Check de más canciones por la puta paginación
        if not result_json['next']:
            break

        # Siguiente petición por la puta paginación
        offset += step

        # Cronómetro
        end_time = time.time()
        elapsed_time = format(end_time - start_time, ".2f")

        print("----- FIN ITERACION " + str(i) + " : " + str(elapsed_time) + "s -----")
        i += 1

    return result_dict


favoritos = canciones_playlist(token, id_playlist)

----- FIN ITERACION 0 : 0.26s -----
----- FIN ITERACION 1 : 0.21s -----
----- FIN ITERACION 2 : 0.28s -----
----- FIN ITERACION 3 : 0.25s -----
----- FIN ITERACION 4 : 0.22s -----
----- FIN ITERACION 5 : 0.28s -----
----- FIN ITERACION 6 : 0.26s -----
----- FIN ITERACION 7 : 0.23s -----
----- FIN ITERACION 8 : 0.22s -----
----- FIN ITERACION 9 : 0.24s -----
----- FIN ITERACION 10 : 0.25s -----
----- FIN ITERACION 11 : 0.24s -----
----- FIN ITERACION 12 : 0.27s -----
----- FIN ITERACION 13 : 0.27s -----
----- FIN ITERACION 14 : 0.29s -----
----- FIN ITERACION 15 : 0.24s -----
----- FIN ITERACION 16 : 0.30s -----
----- FIN ITERACION 17 : 0.29s -----
----- FIN ITERACION 18 : 0.25s -----
----- FIN ITERACION 19 : 0.25s -----
----- FIN ITERACION 20 : 0.29s -----
----- FIN ITERACION 21 : 0.23s -----
----- FIN ITERACION 22 : 0.28s -----
----- FIN ITERACION 23 : 0.25s -----
----- FIN ITERACION 24 : 0.44s -----
----- FIN ITERACION 25 : 0.23s -----
----- FIN ITERACION 26 : 0.24s -----
----- FIN I